In [1]:
# GENERATIVE ADVERSARIAL NETWORK with TF Keras

In [2]:
# source = https://www.datacamp.com/community/tutorials/generative-adversarial-networks

In [3]:
# libraries python
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [4]:
# TF_keras_backend
from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers.core import Dense, Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.datasets import mnist
from keras.optimizers import Adam
from keras import initializers

Using TensorFlow backend.


In [5]:
# Dire à Keras que nous utilisons tensorflow comme moteur principal
os.environ["keras_backend"] = "tensorflow"

# resultats reproductibles
np.random.seed(10)

# La dimension de notre vecteur de bruit 
random_dim = 100

In [6]:
# mnist.load_data() de Keras = import dataset MNIST

In [7]:
# fct data preprocessing

def load_minst_data():
    # charger data
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    
    # normaliser nos data dans [-1, 1]
    x_train = (x_train.astype(np.float32) - 127.5)/127.5
    
    # convertir x_train (60000, 28, 28) to (60000, 784) 784 colonne / ligne
    x_train = x_train.reshape(60000, 784)
    
    return (x_train, y_train, x_test, y_test)

In [8]:
# utilisation de l optimiseur Adam 

# Pour le générateur et le discriminateur, réseau de neurones avec
# 3 couches cachées avec la fonction d'activation étant LeakyReLU . 
# https://keras.io/layers/advanced-activations/

# ajouter des couches dropout pour que le discriminateur améliore sa robustesse vis-à-vis des images invisibles.

In [9]:
# Adam optimizer
def get_optimizer():
    return Adam(lr=0.0002, beta_1=0.5)

# fct generateur
def get_generator(optimizer):
    generator = Sequential()
    generator.add(Dense(256, input_dim=random_dim, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
    generator.add(LeakyReLU(0.2))

    generator.add(Dense(512))
    generator.add(LeakyReLU(0.2))

    generator.add(Dense(1024))
    generator.add(LeakyReLU(0.2))

    generator.add(Dense(784, activation='tanh'))
    generator.compile(loss='binary_crossentropy', optimizer=optimizer)
    return generator

# fct discriminateur
def get_discriminator(optimizer):
    discriminator = Sequential()
    discriminator.add(Dense(1024, input_dim=784, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))

    discriminator.add(Dense(512))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))

    discriminator.add(Dense(256))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))

    discriminator.add(Dense(1, activation='sigmoid'))
    discriminator.compile(loss='binary_crossentropy', optimizer=optimizer)
    return discriminator

In [10]:
# reunir generateur et discriminateur

def get_gan_network(discriminator, random_dim, generator, optimizer):
    
    # Nous avons initialement défini la possibilité d’entraîner sur False, 
    # car nous ne voulons former que le générateur ou discriminateur à la fois
    discriminator.trainable = False
    
    # gan input (noise) = vecteur 100 dimensions
    gan_input = Input(shape=(random_dim,))
    
    # la sortie du générateur = une image
    x = generator(gan_input)
    
    # sortie du discriminateur (proba si l'image est réelle ou non)
    gan_output = discriminator(x)
    gan = Model(inputs=gan_input, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer=optimizer)
    return gan

In [11]:
# créer fonction qui enregistrera vos images générées toutes les 20 Epochs

In [12]:
# Créer un mur d'images MNIST générées

def plot_generated_images(epoch, generator, examples=100, dim=(10, 10), figsize=(10, 10)):
    noise = np.random.normal(0, 1, size=[examples, random_dim])
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(examples, 28, 28)

    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('image_generee_gan_epoch_%d.png' % epoch)

In [13]:
# Vous avez maintenant codé la majorité de votre réseau. 
# Il ne reste plus qu’à former ce réseau et à regarder les images que vous avez créées.

In [14]:
# Après un train de 400 Epochs, on peut voir les images générées. 
# En regardant les images produites après Epoch1 = chiffre tout pourri
# cf = gan_generated_image_epoch_1.png
# En regardant l’image après 40 Epochs, les chiffres commencent à se dessiner et enfin,
# cf = gan_generated_image_epoch_20.png
# cf = gan_generated_image_epoch_40.png

# les images produites après 400 Epochs affichent des chiffres clairs
# nvidia-smi = charge GPU = 42%

In [15]:
def train(epochs=1, batch_size=128):
    
    # charger les data train et de test
    x_train, y_train, x_test, y_test = load_minst_data()
    
    # Diviser les données d'entraînement en lots de taille 128
    batch_count = x_train.shape[0] // batch_size

    # def Gan network
    adam = get_optimizer()
    generator = get_generator(adam)
    discriminator = get_discriminator(adam)
    gan = get_gan_network(discriminator, random_dim, generator, adam)

    for e in range(1, epochs+1):
        print ("-"*15, "Epoch %d" % e, "-"*15)
        for _ in tqdm(range(batch_count)):
            
            # Obtenir 1 ensemble aléatoire de bruits d'entrée et d'images
            noise = np.random.normal(0, 1, size=[batch_size, random_dim])
            image_batch = x_train[np.random.randint(0, x_train.shape[0], size=batch_size)]

            # Générer de fausses images MNIST
            generated_images = generator.predict(noise)
            X = np.concatenate([image_batch, generated_images])

            # Étiquettes pour les données générées et réelles
            y_dis = np.zeros(2*batch_size)
            
            # Lissage unilatéral des étiquettes
            y_dis[:batch_size] = 0.9

            # Train discriminator
            discriminator.trainable = True
            discriminator.train_on_batch(X, y_dis)

            # Train generator
            noise = np.random.normal(0, 1, size=[batch_size, random_dim])
            y_gen = np.ones(batch_size)
            discriminator.trainable = False
            gan.train_on_batch(noise, y_gen)

        if e == 1 or e % 20 == 0:
            plot_generated_images(e, generator)

if __name__ == '__main__':
    train(400, 128)

  0%|          | 0/468 [00:00<?, ?it/s]

--------------- Epoch 1 ---------------


InternalError: failed initializing StreamExecutor for CUDA device ordinal 0: Internal: failed call to cuDevicePrimaryCtxRetain: CUDA_ERROR_OUT_OF_MEMORY: out of memory; total memory reported: 6373572608